In [2]:
import torch
from torchvision.transforms import Compose, functional
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import dill
from pyro.infer import SVI, Trace_ELBO, TraceGraph_ELBO
from pyro.infer import Predictive, Trace_ELBO
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime

### Adding sentiment information

In [25]:
N_COLS = 5
INPUT_SHAPE = [13,5]
INPUT_SIZE = 10
Z_DIM = 2
DATA_PATH ='/home/flor/cs236/data/d_dfs_sent.pkd'
NN_PREDICTIONS_PATH = '/home/flor/cs236/data/predictions_baseline_sent_z2.pkd'
CVAE_PREDICTIONS_PATH = '/home/flor/cs236/data/predictions_sent_z2.pkd'

In [38]:
class STOCK(Dataset):
    def __init__(self, pd_df, input_size = INPUT_SIZE, mask_size=3, n_cols=N_COLS, train = True):
        self.ncols = n_cols
        self.original, self.test_low, self.test_high, self.minmax = self.loadData(pd_df, input_size, mask_size)
        self.transform =  Compose([ToTensor(), MaskData(pos=mask_size)])
        self.train = train
    def __len__(self):
        return len(self.original)

    def __getitem__(self, item):
        sample = {"original":self.original[item]}
        test_low = {"original":self.test_low[0]}
        test_high = {"original":self.test_high[0]}
        if self.transform:
            sample = self.transform(sample)
            test_low = self.transform(test_low)
            test_high = self.transform(test_high)
        if self.train:
            return sample
        else:
            return {'input_low': test_low['input'], 'input_high': test_high['input']}
    
    def loadData(self, df, input_size, mask_size):
    
        minmax = MinMaxScaler().fit(df[['close','high','low']].values.reshape(-1,3))
        df_norm = pd.DataFrame(minmax.transform(df[['close','high','low']].values.reshape(-1,3)))
        
        df_norm.index = df.index
        df_norm.columns = ['close','high','low']
   
        add_cols = []
        if self.ncols > 3:
            add_cols = ['relevance_score','ticker_sentiment_score' ]
            for col in add_cols:
                df_norm[col] = df[col]

        dataset = []
        for i in range(len(df_norm) - (input_size+mask_size) - 1):
            dataset.append(df_norm.iloc[i:i + (input_size+mask_size),:])
        
        testset_low = [pd.concat([df_norm.iloc[-(input_size+mask_size-1) : -1],pd.DataFrame(
            [dict(zip(df_norm.columns,[0]*len(df_norm.columns)))]*2)])]
        
        testset_high = [pd.concat([df_norm.iloc[-input_size : ]
                        ,pd.DataFrame([dict(zip(df_norm.columns
                                                ,[0]*len(df_norm.columns)))]*mask_size)])]

        return dataset, testset_low, testset_high, minmax
    
    def reverseMinMax(self, pred):
        return self.minmax.inverse_transform(pred)

class ToTensor:
    def __call__(self, sample):
        sample['original'] = torch.from_numpy(sample['original'].values).float()
        return sample

class MaskData:
    """This transformation masks the values to be predicted with -1 and
    adds the target output in the sample dict as the complementary of the input
    """

    def __init__(self, pos=1, mask_with=-1, ncols= N_COLS):
        self.mask_with = mask_with
        self.pos = -pos
        self.ncols = ncols

    def __call__(self, sample):
        tensor = sample['original']
        inp = tensor.detach().clone()

        # remove the last one
        inp[self.pos:] = self.mask_with

        # now, sets the input as complementary
        out = tensor.clone()
        out[inp != -1] = self.mask_with

        sample["input"] = inp
        sample["output"] = out
        
        # Remove the sentiment from output so that we dont calculate loss for it.
        out[self.pos:, 3:self.ncols] = -1
        return sample


In [39]:
pd.read_pickle(DATA_PATH)['COST'][datetime(2022,10,27)].tail(15)

,close,high,low,relevance_score,ticker_sentiment_score
date,,,,,
2022-10-07,468.15,479.1050,466.1200,0.485711,0.140444
2022-10-10,466.31,470.2500,461.6100,0.431765,0.129190
2022-10-11,472.02,477.2500,463.7000,0.901780,0.335862
2022-10-12,466.38,479.7400,466.3000,0.645516,0.345227
2022-10-13,467.99,471.7400,449.0300,0.159569,0.207263
2022-10-14,454.65,473.3900,453.7900,0.381949,0.124561
2022-10-17,464.17,466.8799,460.4800,0.895116,0.317930
2022-10-18,473.27,476.5205,468.8300,0.706550,0.352159
2022-10-19,471.43,474.8667,466.9500,0.980595,0.386944


In [40]:
def get_data(df, batch_size=16):
    data, dl, ds = {}, {}, {}
    data['train'] = STOCK(df)
    data['test'] = STOCK(df, train = False)
    dl['train'] = DataLoader(data['train'], batch_size=batch_size, shuffle=True, num_workers=0)
    dl['val'] = DataLoader(data['train'], batch_size=batch_size, shuffle=False, num_workers=0)
    ds['train'] = ds['val'] = len(dl['train'])
    return data, dl, ds

def get_all_data(filepath = DATA_PATH):
    datasets, dataloaders, dataset_sizes = defaultdict(dict), defaultdict(dict), defaultdict(dict)
    d_all = pd.read_pickle(filepath)
    for ticker, dates in d_all.items():    
        for date,df in dates.items():
            datasets[ticker][date], dataloaders[ticker][date], dataset_sizes[ticker][date] = get_data(df)
    return datasets, dataloaders, dataset_sizes
datasets, dataloaders, dataset_sizes = get_all_data()

In [41]:
datasets['COST'][datetime(2022,10,27)]['train'].original

[               close      high       low  relevance_score  \
 date                                                        
 2021-10-29  0.391974  0.356123  0.419161              0.0   
 2021-11-01  0.393696  0.360056  0.421985              0.0   
 2021-11-02  0.420415  0.385127  0.450853              0.0   
 2021-11-03  0.448283  0.404045  0.466700              0.0   
 2021-11-04  0.517639  0.498092  0.520357              0.0   
 2021-11-05  0.504592  0.485498  0.534058              0.0   
 2021-11-08  0.456007  0.461018  0.489972              0.0   
 2021-11-09  0.481578  0.437520  0.510158              0.0   
 2021-11-10  0.464878  0.444380  0.508616              0.0   
 2021-11-11  0.499687  0.456057  0.510054              0.0   
 2021-11-12  0.525728  0.495093  0.558115              0.0   
 2021-11-15  0.539923  0.513848  0.583113              0.0   
 2021-11-16  0.575566  0.542198  0.579975              0.0   
 
             ticker_sentiment_score  
 date                         

In [42]:
datasets['COST'][datetime(2022, 10, 27)]['train'][-1]

{'original': tensor([[0.2699, 0.2740, 0.3117, 0.4857, 0.1404],
         [0.2603, 0.2257, 0.2882, 0.4318, 0.1292],
         [0.2901, 0.2639, 0.2991, 0.9018, 0.3359],
         [0.2607, 0.2775, 0.3127, 0.6455, 0.3452],
         [0.2691, 0.2338, 0.2224, 0.1596, 0.2073],
         [0.1995, 0.2428, 0.2473, 0.3819, 0.1246],
         [0.2491, 0.2073, 0.2822, 0.8951, 0.3179],
         [0.2966, 0.2599, 0.3259, 0.7066, 0.3522],
         [0.2870, 0.2509, 0.3161, 0.9806, 0.3869],
         [0.2515, 0.2357, 0.2950, 0.4866, 0.1379],
         [0.3223, 0.2765, 0.2917, 0.7469, 0.5883],
         [0.4203, 0.3770, 0.3896, 0.8390, 0.4128],
         [0.4312, 0.3889, 0.4392, 0.6313, 0.3158]]),
 'input': tensor([[ 0.2699,  0.2740,  0.3117,  0.4857,  0.1404],
         [ 0.2603,  0.2257,  0.2882,  0.4318,  0.1292],
         [ 0.2901,  0.2639,  0.2991,  0.9018,  0.3359],
         [ 0.2607,  0.2775,  0.3127,  0.6455,  0.3452],
         [ 0.2691,  0.2338,  0.2224,  0.1596,  0.2073],
         [ 0.1995,  0.2428,  0.247

In [43]:
datasets['COST'][datetime(2022, 10, 27)]['test'][0]

{'input_low': tensor([[ 0.2607,  0.2775,  0.3127,  0.6455,  0.3452],
         [ 0.2691,  0.2338,  0.2224,  0.1596,  0.2073],
         [ 0.1995,  0.2428,  0.2473,  0.3819,  0.1246],
         [ 0.2491,  0.2073,  0.2822,  0.8951,  0.3179],
         [ 0.2966,  0.2599,  0.3259,  0.7066,  0.3522],
         [ 0.2870,  0.2509,  0.3161,  0.9806,  0.3869],
         [ 0.2515,  0.2357,  0.2950,  0.4866,  0.1379],
         [ 0.3223,  0.2765,  0.2917,  0.7469,  0.5883],
         [ 0.4203,  0.3770,  0.3896,  0.8390,  0.4128],
         [ 0.4312,  0.3889,  0.4392,  0.6313,  0.3158],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000]]),
 'input_high': tensor([[ 0.1995,  0.2428,  0.2473,  0.3819,  0.1246],
         [ 0.2491,  0.2073,  0.2822,  0.8951,  0.3179],
         [ 0.2966,  0.2599,  0.3259,  0.7066,  0.3522],
         [ 0.2870,  0.2509,  0.3161,  0.9806,  0.3869],
         [ 0.2515, 

In [44]:
import copy
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm


class BaselineNet(nn.Module):
    def __init__(self, hidden_1, hidden_2, input_shape=INPUT_SHAPE):
        super().__init__()
        self.fc1 = nn.Linear(np.prod(input_shape), hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, np.prod(input_shape))
        self.relu = nn.ReLU()
        self.input_shape = input_shape

    def forward(self, x):
        x = x.view(-1, np.prod(self.input_shape))
        hidden = self.relu(self.fc1(x))
        hidden = self.relu(self.fc2(hidden))
        y = torch.sigmoid(self.fc3(hidden))
        return y


class MaskedMSELoss(nn.Module):
    def __init__(self, masked_with=-1):
        super().__init__()
        self.masked_with = masked_with

    def forward(self, input, target):
        target = target.view(input.shape)
        loss = F.mse_loss(input, target, reduction="none")
        loss[target == self.masked_with] = 0
        return loss.sum()


def train_baseline(
    device,
    dataloaders,
    dataset_sizes,
    learning_rate,
    num_epochs,
    early_stop_patience,
    model_path=None,
    input_shape = INPUT_SHAPE
):
    # Train baseline
    baseline_net = BaselineNet(500, 500,input_shape=input_shape)
    baseline_net.to(device)
    optimizer = torch.optim.Adam(baseline_net.parameters(), lr=learning_rate)
    criterion = MaskedMSELoss()
    best_loss = np.inf
    early_stop_count = 0

    for epoch in range(num_epochs):
        for phase in ["train", "val"]:
            if phase == "train":
                baseline_net.train()
            else:
                baseline_net.eval()

            running_loss = 0.0
            num_preds = 0

            #bar = tqdm(dataloaders[phase], desc="NN Epoch {} {}".format(epoch, phase).ljust(20))

            #for i, batch in enumerate(bar):
            for batch in dataloaders[phase]:#don't want 100 epoch bars for each of hundreds of models
                inputs = batch["input"].to(device)
                outputs = batch["output"].to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    preds = baseline_net(inputs)
                    loss = criterion(preds, outputs) / inputs.size(0)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                num_preds += 1
                #if i % 10 == 0:
                #    bar.set_postfix(
                #        loss="{:.2f}".format(running_loss / num_preds),
                #        early_stop_count=early_stop_count,
                #    )

            epoch_loss = running_loss / dataset_sizes[phase]
            # deep copy the model
            if phase == "val":
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(baseline_net.state_dict())
                    early_stop_count = 0
                else:
                    early_stop_count += 1

        if early_stop_count >= early_stop_patience:
            break

    baseline_net.load_state_dict(best_model_wts)
    baseline_net.eval()

    # Save model weights
    #Path(model_path).parent.mkdir(parents=True, exist_ok=True)
    #torch.save(baseline_net.state_dict(), model_path)

    return baseline_net


In [46]:
baseline_net = train_baseline(device=torch.device(
        "cuda:0" if torch.cuda.is_available() and args.cuda else "cpu"),
            dataloaders=dataloaders['COST'][datetime(2022, 10, 27)],
            dataset_sizes=dataset_sizes['COST'][datetime(2022, 10, 27)],
            learning_rate=1.0e-3,
            num_epochs=101,
            early_stop_patience=20,
            model_path="baseline_net.pth")

In [47]:
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, TraceGraph_ELBO


class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_1, hidden_2, input_shape=INPUT_SHAPE):
        super().__init__()
        self.fc1 = nn.Linear(np.prod(input_shape), hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc31 = nn.Linear(hidden_2, z_dim)
        self.fc32 = nn.Linear(hidden_2, z_dim)
        self.relu = nn.ReLU()
        self.input_shape = input_shape

    def forward(self, x, y):
        # put x and y together in the same tensor for simplification
        xc = x.clone()
        xc[x == -1] = y[x == -1]
        xc = xc.view(-1, np.prod(self.input_shape))
        # then compute the hidden units
        hidden = self.relu(self.fc1(xc))
        hidden = self.relu(self.fc2(hidden))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc31(hidden)
        z_scale = torch.exp(self.fc32(hidden))
        return z_loc, z_scale


class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_1, hidden_2,  input_shape=INPUT_SHAPE):
        super().__init__()
        self.fc1 = nn.Linear(z_dim, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, np.prod(input_shape))
        self.relu = nn.ReLU()

    def forward(self, z):
        y = self.relu(self.fc1(z))
        y = self.relu(self.fc2(y))
        y = torch.sigmoid(self.fc3(y))
        return y


class CVAE(nn.Module):
    def __init__(self, z_dim, hidden_1, hidden_2, pre_trained_baseline_net,  input_shape=INPUT_SHAPE):
        super().__init__()
        # The CVAE is composed of multiple MLPs, such as recognition network
        # qφ(z|x, y), (conditional) prior network pθ(z|x), and generation
        # network pθ(y|x, z). Also, CVAE is built on top of the NN: not only
        # the direct input x, but also the initial guess y_hat made by the NN
        # are fed into the prior network.
        self.baseline_net = pre_trained_baseline_net
        self.prior_net = Encoder(z_dim, hidden_1, hidden_2)
        self.generation_net = Decoder(z_dim, hidden_1, hidden_2)
        self.recognition_net = Encoder(z_dim, hidden_1, hidden_2)
        self.input_shape = input_shape

    def model(self, xs, ys=None):
        # register this pytorch module and all of its sub-modules with pyro
        pyro.module("generation_net", self)
        batch_size = xs.shape[0]

        with pyro.plate("data"):
            # Prior network uses the baseline predictions as initial guess.
            # This is the generative process with recurrent connection
            with torch.no_grad():
                # this ensures the training process does not change the
                # baseline network
                y_hat = self.baseline_net(xs).view(xs.shape)

            # sample the handwriting style from the prior distribution, which is
            # modulated by the input xs.
            prior_loc, prior_scale = self.prior_net(xs, y_hat)
            zs = pyro.sample("z", dist.Normal(prior_loc, prior_scale).to_event(1))

            # the output y is generated from the distribution pθ(y|x, z)
            loc = self.generation_net(zs)

            if ys is not None:
                # In training, we will only sample in the masked image
                mask_loc = loc[(xs == -1).view(-1, np.prod(self.input_shape))].view(batch_size, -1)
                mask_ys = ys[xs == -1].view(batch_size, -1)
                
                pyro.deterministic("y", loc)

            else:
                # In testing, no need to sample: the output is already a
                # probability in [0, 1] range, which better represent pixel
                # values considering grayscale. If we sample, we will force
                # each pixel to be  either 0 or 1, killing the grayscale
                pyro.deterministic("y", loc.detach())

            # return the loc so we can visualize it later
            return loc

    def guide(self, xs, ys=None):
        with pyro.plate("data"):
            if ys is None:
                # at inference time, ys is not provided. In that case,
                # the model uses the prior network
                y_hat = self.baseline_net(xs).view(xs.shape)
                loc, scale = self.prior_net(xs, y_hat)
            else:
                # at training time, uses the variational distribution
                # q(z|x,y) = normal(loc(x,y),scale(x,y))
                loc, scale = self.recognition_net(xs, ys)
            
            pyro.sample("z", dist.Normal(loc, scale).to_event(1))


def train_cvae(
    device,
    dataloaders,
    dataset_sizes,
    learning_rate,
    num_epochs,
    early_stop_patience,
    model_path="cvae_net1.pth",
    pre_trained_baseline_net=baseline_net,
):
    # clear param store
    pyro.clear_param_store()

    cvae_net = CVAE(Z_DIM, 500, 500, pre_trained_baseline_net)
    cvae_net.to(device)
    optimizer = pyro.optim.Adam({"lr": learning_rate})
    svi = SVI(cvae_net.model, cvae_net.guide, optimizer, loss=Trace_ELBO())

    best_loss = np.inf
    early_stop_count = 0
    Path(model_path).parent.mkdir(parents=True, exist_ok=True)
    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            running_loss = 0.0
            num_preds = 0

            # Iterate over data.
            #bar = tqdm(
            #    dataloaders[phase],
            #    desc="CVAE Epoch {} {}".format(epoch, phase).ljust(20))
            #for i,batch in enumerate(bar):
            for batch in dataloaders[phase]:#don't want epoch bars for hundreds of models
                inputs = batch["input"].to(device)
                outputs = batch["output"].to(device)

                if phase == "train":
                    loss = svi.step(inputs, outputs)
                else:
                    loss = svi.evaluate_loss(inputs, outputs)

                # statistics
                running_loss += loss / inputs.size(0)
                num_preds += 1
                #if i % 10 == 0:
                #    bar.set_postfix(
                #        loss="{:.2f}".format(running_loss / num_preds),
                #        early_stop_count=early_stop_count)

            epoch_loss = running_loss / dataset_sizes[phase]
            # deep copy the model
            if phase == "val":
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    torch.save(cvae_net.state_dict(), model_path)
                    early_stop_count = 0
                else:
                    early_stop_count += 1

        if early_stop_count >= early_stop_patience:
            break

    # Save model weights
    cvae_net.load_state_dict(torch.load(model_path))
    cvae_net.eval()
    return cvae_net

cvae_net = train_cvae(
            device=torch.device(
        "cuda:0" if torch.cuda.is_available() and args.cuda else "cpu"),
            dataloaders=dataloaders['COST'][datetime(2022, 10, 27)],
            dataset_sizes=dataset_sizes['COST'][datetime(2022, 10, 27)],
            learning_rate=1.0e-3,
            num_epochs=101,
            early_stop_patience=20,
            model_path="cvae_net1.pth",
            pre_trained_baseline_net=baseline_net,
        )


In [48]:
def train_predict(ticker, date
        , device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), mask_shape=[INPUT_SIZE,3]):
    d_baseline, d_cvae = {}, {}
    dataset = datasets[ticker][date]
    dataloader_test = DataLoader(dataset["test"], batch_size=1, shuffle=False)
    batch = next(iter(dataloader_test))
    
    def get_pred(model, input_type, prelim_high_prediction=False):
        ### input_type: input_low or input_high
        prediction = model(batch[input_type].to(device))
        if type(prediction) == dict:
            prediction = prediction['y']
        prediction = prediction.reshape(-1,N_COLS).detach()
        
        prediction = torch.from_numpy(dataset['test'].reverseMinMax(prediction[:,:3]))
        index = {'input_low':2, 'input_high':1}
        if input_type == 'input_low' and prelim_high_prediction:
            return prediction[mask_shape[0], index['input_low']].item()\
                    , prediction[mask_shape[0]+mask_shape[1]-1, index['input_high']].item()
        return prediction[mask_shape[0], index[input_type]].item(),0
    
    baseline_net = train_baseline(device=device,
            dataloaders=dataloaders[ticker][date],
            dataset_sizes=dataset_sizes[ticker][date],
            learning_rate=1.0e-3,
            num_epochs=101,
            early_stop_patience=20, input_shape=(sum(mask_shape), N_COLS))

    d_baseline['low'], d_baseline['estimated_high'] = get_pred(baseline_net, 'input_low', prelim_high_prediction = True)
    d_baseline['high'] = get_pred(baseline_net, 'input_high')[0]
    
    cvae_net = train_cvae(device,
            dataloaders=dataloaders[ticker][date],
            dataset_sizes=dataset_sizes[ticker][date],
            learning_rate=1.0e-3,
            num_epochs=101,
            early_stop_patience=20,
            pre_trained_baseline_net=baseline_net,)
    predictive = Predictive(cvae_net.model, guide=cvae_net.guide, num_samples=1)

    d_cvae['low'], d_cvae['estimated_high'] = get_pred(predictive, 'input_low' , prelim_high_prediction = True)
    d_cvae['high'] = get_pred(predictive, 'input_high')[0]

    return d_baseline,d_cvae

def get_predictions():
    d_all = pd.read_pickle(DATA_PATH)
    d_baseline, d_cvae = defaultdict(dict), defaultdict(dict)
    for ticker, dates in d_all.items():
        for date,df in dates.items():
            print(ticker,date)
            d_baseline[ticker][date], d_cvae[ticker][date] = train_predict(ticker, date)
        dill.dump(d_baseline,open(NN_PREDICTIONS_PATH,'wb'))
        dill.dump(d_cvae,open(CVAE_PREDICTIONS_PATH,'wb'))
    return d_baseline, d_cvae

get_predictions()


COST 2022-10-27 00:00:00
COST 2023-02-02 00:00:00
COST 2023-05-04 00:00:00
COST 2023-08-24 00:00:00
COST 2023-11-02 00:00:00
CE 2022-10-28 00:00:00
CE 2023-07-28 00:00:00
CE 2023-10-27 00:00:00
AON 2022-10-31 00:00:00
AON 2023-01-31 00:00:00
AON 2023-04-28 00:00:00
AON 2023-07-31 00:00:00
AON 2023-10-31 00:00:00
MKTX 2022-11-01 00:00:00
MKTX 2023-02-07 00:00:00
MKTX 2023-05-09 00:00:00
MKTX 2023-08-01 00:00:00
CAG 2022-11-02 00:00:00
CAG 2023-11-01 00:00:00
NSC 2022-11-03 00:00:00
NSC 2023-08-03 00:00:00
AMP 2022-11-04 00:00:00
AMP 2023-02-09 00:00:00
AMP 2023-08-04 00:00:00
AWK 2022-11-07 00:00:00
AWK 2023-02-06 00:00:00
AWK 2023-05-08 00:00:00
AWK 2023-08-07 00:00:00
WST 2022-11-08 00:00:00
WST 2023-01-24 00:00:00
WST 2023-04-25 00:00:00
WST 2023-07-25 00:00:00
POOL 2022-11-09 00:00:00
POOL 2023-05-16 00:00:00
POOL 2023-08-09 00:00:00
GWW 2022-11-10 00:00:00
GWW 2023-02-10 00:00:00
GWW 2023-05-05 00:00:00
GWW 2023-08-11 00:00:00
TFX 2022-11-14 00:00:00
TFX 2023-03-02 00:00:00
TFX 202

(defaultdict(dict,
             {'COST': {datetime.datetime(2022, 10, 27, 0, 0): {'low': 483.4971090748459,
                'estimated_high': 493.8318836188317,
                'high': 500.6296461063624},
               datetime.datetime(2023, 2, 2, 0, 0): {'low': 505.2399988534689,
                'estimated_high': 511.7678098458052,
                'high': 513.8853504663706},
               datetime.datetime(2023, 5, 4, 0, 0): {'low': 485.1542407323778,
                'estimated_high': 489.0165546834469,
                'high': 493.81766644120216},
               datetime.datetime(2023, 8, 24, 0, 0): {'low': 545.8794462919235,
                'estimated_high': 554.021239567995,
                'high': 549.4480869269371},
               datetime.datetime(2023, 11, 2, 0, 0): {'low': 550.0604794740676,
                'estimated_high': 559.1065002864599,
                'high': 559.3234035611152}},
              'CE': {datetime.datetime(2022, 10, 28, 0, 0): {'low': 92.442113917917,
   

In [28]:
def visualize(
    device,
    pre_trained_baseline,
    pre_trained_cvae,
    num_images,
    image_path=None,
    ticker='COST',
    date=datetime(2022, 10, 27)
):
    # Load sample random data
    datasets, _, dataset_sizes = get_data(pd.read_pickle('d_dfs_sent.pkd')[ticker][date])
    dataloader = DataLoader(datasets["train"], batch_size=num_images, shuffle=False)

    batch = next(iter(dataloader))
    inputs = batch["input"].to(device)
    outputs = batch["output"].to(device)
    actual = batch["original"].to(device)

    # Make predictions
    with torch.no_grad():
        baseline_preds = pre_trained_baseline(inputs).view(outputs.shape)

    predictive = Predictive(
        pre_trained_cvae.model, guide=pre_trained_cvae.guide, num_samples=1
    )

    cvae_preds = predictive(inputs)["y"].view(num_images, 8, 5)
    
    baseline_preds = torch.from_numpy(datasets['train'].\
                                      reverseMinMax(baseline_preds.reshape(-1,5))).reshape(outputs.shape)
    actual = torch.from_numpy(datasets['train'].\
                              reverseMinMax(actual.reshape(-1,5))).reshape(outputs.shape)
    cvae_preds = torch.from_numpy(datasets['train'].\
                                  reverseMinMax(cvae_preds.reshape(-1,3))).reshape(outputs.shape)

    # Predictions are only made on the next day, so that is collected into
    # a single series.
 
    actual_final = actual[:,5]
    baseline_final = baseline_preds[:,5]
    cvae_final = cvae_preds[:,5]
    
    t = range(baseline_final.shape[0])

    fig, axs = plt.subplots(3)

    for i in range(actual_final.shape[-1]):
        axs[i].plot(t, actual_final[:,i])
        axs[i].plot(t, baseline_final[:,i])
        axs[i].plot(t, cvae_final[:,i]) # Assumes only one sample

    axs[0].title.set_text('Close')
    axs[1].title.set_text('High')
    axs[2].title.set_text('Low')

    labels = ['actual', 'baseline', 'cvae']
    axs[0].legend(labels=labels)
    
    # Save plot
    fig.savefig(image_path)
    fig.clf()

test = visualize(device=torch.device(
        "cuda:0" if torch.cuda.is_available() and args.cuda else "cpu"),
            pre_trained_baseline=baseline_net,
            pre_trained_cvae=cvae_net,
            num_images=10,
            image_path="cvae_plot.png")
test

C:\Users\Flor\AppData\Local\Temp\ipykernel_5072\3513850043.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  testset_high = [df_norm.iloc[-input_size : ].append(
C:\Users\Flor\AppData\Local\Temp\ipykernel_5072\3513850043.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  testset_high = [df_norm.iloc[-input_size : ].append(


ValueError: operands could not be broadcast together with shapes (80,5) (3,) (80,5) 

In [35]:
torch.ones(10,1)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [40]:
len(torch.ones(10).unsqueeze(1).size())

2

In [39]:
torch.ones(10).shape

torch.Size([10])